# 第09章：Retrievers（检索器深入）

## 学习目标

本章将深入学习：
1. Retriever接口和Runnable协议
2. VectorStoreRetriever的不同检索策略
3. similarity、MMR、similarity_score_threshold
4. MultiQueryRetriever（多查询检索）
5. ContextualCompressionRetriever（上下文压缩）
6. ParentDocumentRetriever（父文档检索）
7. 自定义Retriever
8. 检索策略对比和选择

## 什么是Retriever？

Retriever是一个通用的检索接口：
- 输入：字符串查询
- 输出：Document列表
- 比Vector Store更通用（可以连接API、数据库等）
- 是Runnable，支持invoke、batch、stream等方法

## 为什么重要？

检索质量直接影响RAG效果！不同的检索策略适用于不同场景，选择合适的Retriever是优化RAG性能的关键。

In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

# 配置Embedding模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL
)

print("环境配置完成！")
print(f"Chat模型: {model.model_name}")
print(f"Embedding模型: {embeddings.model}")

环境配置完成！
Chat模型: gpt-4.1-mini
Embedding模型: text-embedding-3-small


In [2]:
### 1.1 准备测试数据

print("【准备测试数据】")
print()

# 创建多样化的文档集
documents = [
    Document(page_content="Python是一种高级编程语言，以简洁的语法著称。python是由top开发", metadata={"topic": "Python", "type": "intro"}),
    Document(page_content="Python广泛应用于数据科学、机器学习和Web开发。", metadata={"topic": "Python", "type": "application"}),
    Document(page_content="Java是一种面向对象的编程语言，具有跨平台特性。", metadata={"topic": "Java", "type": "intro"}),
    Document(page_content="Java常用于企业级应用开发和Android应用开发。", metadata={"topic": "Java", "type": "application"}),
    Document(page_content="JavaScript是Web开发的核心语言，运行在浏览器中。", metadata={"topic": "JavaScript", "type": "intro"}),
    Document(page_content="JavaScript可以用于前端和后端开发（Node.js）。", metadata={"topic": "JavaScript", "type": "application"}),
    Document(page_content="机器学习是人工智能的一个分支，让计算机从数据中学习。", metadata={"topic": "AI", "type": "concept"}),
    Document(page_content="深度学习使用神经网络处理复杂的模式识别任务。", metadata={"topic": "AI", "type": "concept"}),
    Document(page_content="LangChain是构建LLM应用的框架，支持RAG和Agent。", metadata={"topic": "LangChain", "type": "tool"}),
    Document(page_content="RAG技术结合检索和生成，让AI能回答私有数据问题。", metadata={"topic": "RAG", "type": "concept"}),
]

# 创建向量存储
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents)

print(f"已索引 {len(documents)} 个文档")
print(f"主题分布: {set(doc.metadata['topic'] for doc in documents)}")

【准备测试数据】

已索引 10 个文档
主题分布: {'JavaScript', 'LangChain', 'RAG', 'AI', 'Java', 'Python'}


## 2. VectorStoreRetriever基础

### 检索策略类型

VectorStoreRetriever支持三种主要的search_type：
1. **similarity**：相似度检索（默认）
2. **mmr**：最大边际相关性（平衡相关性和多样性）
3. **similarity_score_threshold**：阈值过滤（只返回超过阈值的结果）

In [3]:
### 2.1 Similarity检索（相似度检索）

print("【Similarity检索】")
print()

# 创建similarity retriever
retriever_similarity = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

query = "Python编程"
docs = retriever_similarity.invoke(query)

print(f"查询: {query}")
print(f"检索到 {len(docs)} 个文档:\n")
for i, doc in enumerate(docs, 1):
    print(f"{i}. {doc.page_content}")
    print(f"   主题: {doc.metadata['topic']}")
    print()

【Similarity检索】

查询: Python编程
检索到 3 个文档:

1. Python是一种高级编程语言，以简洁的语法著称。python是由top开发
   主题: Python

2. Python广泛应用于数据科学、机器学习和Web开发。
   主题: Python

3. Java是一种面向对象的编程语言，具有跨平台特性。
   主题: Java



### 2.2 MMR检索（最大边际相关性）

MMR平衡相关性和多样性，避免返回过于相似的文档。

**工作原理**：
1. 先检索更多候选文档（fetch_k）
2. 从中选择k个，既相关又多样

**参数**：
- `k`: 最终返回的文档数
- `fetch_k`: 初始检索的候选文档数
- `lambda_mult`: 多样性权重（0=最大多样性，1=最大相关性）

In [4]:
print("【MMR检索对比】")
print()

# MMR retriever
retriever_mmr = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,
        "fetch_k": 10,
        "lambda_mult": 0.5  # 平衡相关性和多样性
    }
)

query = "编程语言"
docs_mmr = retriever_mmr.invoke(query)

print(f"查询: {query}")
print("\nMMR结果（相关且多样）:")
for i, doc in enumerate(docs_mmr, 1):
    print(f"{i}. [{doc.metadata['topic']}] {doc.page_content[:40]}...")

print("\n" + "="*60)

# 对比：普通similarity可能返回相似的文档
docs_sim = retriever_similarity.invoke(query)
print("\nSimilarity结果（可能重复相似）:")
for i, doc in enumerate(docs_sim, 1):
    print(f"{i}. [{doc.metadata['topic']}] {doc.page_content[:40]}...")

【MMR检索对比】

查询: 编程语言

MMR结果（相关且多样）:
1. [JavaScript] JavaScript是Web开发的核心语言，运行在浏览器中。...
2. [Python] Python是一种高级编程语言，以简洁的语法著称。python是由top开发...
3. [AI] 机器学习是人工智能的一个分支，让计算机从数据中学习。...


Similarity结果（可能重复相似）:
1. [JavaScript] JavaScript是Web开发的核心语言，运行在浏览器中。...
2. [Python] Python是一种高级编程语言，以简洁的语法著称。python是由top开发...
3. [Java] Java是一种面向对象的编程语言，具有跨平台特性。...


### 2.3 Similarity Score Threshold（阈值过滤）

只返回相似度分数超过阈值的文档，过滤掉不相关的结果。

In [5]:
print("【阈值过滤检索】")
print()

from langchain_community.vectorstores import Chroma

vector_store = Chroma.from_documents(documents, embeddings)

# 创建带阈值的retriever
retriever_threshold = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.35,  # 只返回分数>0.35的文档
        "k": 5
    }
)

# 测试相关查询
query1 = "Python数据科学"
docs1 = retriever_threshold.invoke(query1)
print(f"相关查询: {query1}")
print(f"返回 {len(docs1)} 个文档（分数>0.35）\n")

# 测试不太相关的查询
query2 = "量子计算"
docs2 = retriever_threshold.invoke(query2)
print(f"不相关查询: {query2}")
print(f"返回 {len(docs2)} 个文档（可能为0，因为没有相关文档）")

【阈值过滤检索】



/home/iip/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1043: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'topic': 'Python', 'type': 'application'}, page_content='Python广泛应用于数据科学、机器学习和Web开发。'), 0.3531092220710885), (Document(metadata={'type': 'concept', 'topic': 'RAG'}, page_content='RAG技术结合检索和生成，让AI能回答私有数据问题。'), 0.12248710396293427), (Document(metadata={'topic': 'AI', 'type': 'concept'}, page_content='机器学习是人工智能的一个分支，让计算机从数据中学习。'), 0.11873106111734044), (Document(metadata={'topic': 'Python', 'type': 'intro'}, page_content='Python是一种高级编程语言，以简洁的语法著称。python是由top开发'), 0.11648092516904085), (Document(metadata={'type': 'concept', 'topic': 'AI'}, page_content='深度学习使用神经网络处理复杂的模式识别任务。'), -0.007342216777417754)]
  self.vectorstore.similarity_search_with_relevance_scores(


相关查询: Python数据科学
返回 1 个文档（分数>0.35）



/home/iip/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1043: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'topic': 'Java', 'type': 'application'}, page_content='Java常用于企业级应用开发和Android应用开发。'), -0.14985630592669952), (Document(metadata={'type': 'concept', 'topic': 'RAG'}, page_content='RAG技术结合检索和生成，让AI能回答私有数据问题。'), -0.15153695365791986), (Document(metadata={'topic': 'JavaScript', 'type': 'intro'}, page_content='JavaScript是Web开发的核心语言，运行在浏览器中。'), -0.15314890202606746), (Document(metadata={'type': 'tool', 'topic': 'LangChain'}, page_content='LangChain是构建LLM应用的框架，支持RAG和Agent。'), -0.1555283445055986), (Document(metadata={'topic': 'Python', 'type': 'application'}, page_content='Python广泛应用于数据科学、机器学习和Web开发。'), -0.16283171900296023)]
  self.vectorstore.similarity_search_with_relevance_scores(
No relevant docs were retrieved using the relevance score threshold 0.35


不相关查询: 量子计算
返回 0 个文档（可能为0，因为没有相关文档）


In [6]:
print("【MultiQueryRetriever】")
print()

from langchain_classic.retrievers.multi_query import MultiQueryRetriever

# 创建MultiQueryRetriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    llm=model
)

# 设置日志查看生成的查询
import logging
logging.basicConfig()
logging.getLogger("langchain_classic.retrievers.multi_query").setLevel(logging.INFO)

query = "如何学习编程？"
print(f"原始查询: {query}\n")

docs = multi_query_retriever.invoke(query)
print(f"\n检索到 {len(docs)} 个去重后的文档:")
for i, doc in enumerate(docs, 1):
    print(f"{i}. [{doc.metadata['topic']}] {doc.page_content[:50]}...")

【MultiQueryRetriever】

原始查询: 如何学习编程？



INFO:langchain_classic.retrievers.multi_query:Generated queries: ['学习编程有哪些有效的方法和步骤？  ', '初学者应该如何系统地掌握编程技能？  ', '有哪些适合新手的编程学习资源和技巧？']



检索到 4 个去重后的文档:
1. [Python] Python是一种高级编程语言，以简洁的语法著称。python是由top开发...
2. [Python] Python广泛应用于数据科学、机器学习和Web开发。...
3. [Java] Java是一种面向对象的编程语言，具有跨平台特性。...
4. [JavaScript] JavaScript可以用于前端和后端开发（Node.js）。...


## 4. ContextualCompressionRetriever（上下文压缩）

压缩检索到的文档，只保留与查询最相关的部分，减少噪音。

**优势**：
- 减少传给LLM的token数
- 提高答案质量
- 降低成本

In [7]:
print("【上下文压缩检索】")
print()

from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

# 创建压缩器
compressor = LLMChainExtractor.from_llm(model)

# 创建压缩检索器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_store.as_retriever(search_kwargs={"k": 3})
)

query = "Python用于什么？"
print(f"查询: {query}\n")

# 普通检索
normal_docs = vector_store.as_retriever().invoke(query)
print("普通检索结果（完整文档）:")
for i, doc in enumerate(normal_docs[:2], 1):
    print(f"{i}. {doc.page_content}")
print()

# 压缩检索
compressed_docs = compression_retriever.invoke(query)
print("压缩检索结果（只保留相关部分）:")
for i, doc in enumerate(compressed_docs[:2], 1):
    print(f"{i}. {doc.page_content}")

【上下文压缩检索】

查询: Python用于什么？

普通检索结果（完整文档）:
1. Python是一种高级编程语言，以简洁的语法著称。python是由top开发
2. Python广泛应用于数据科学、机器学习和Web开发。

压缩检索结果（只保留相关部分）:
1. Python是一种高级编程语言，以简洁的语法著称。
2. Python广泛应用于数据科学、机器学习和Web开发。


## 5. ParentDocumentRetriever（父文档检索）

检索小块文档（精确匹配），但返回完整的父文档（保留上下文）。

**适用场景**：
- 需要精确检索但也需要完整上下文
- 文档有层级结构

In [ ]:
print("【父文档检索】")
print()

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 创建父文档
parent_docs = [
    Document(
        page_content="Python简介：Python是一种高级编程语言。它的设计哲学强调代码可读性。Python支持多种编程范式。",
        metadata={"source": "python_guide"}
    ),
    Document(
        page_content="Java简介：Java是面向对象的语言。它具有跨平台特性。Java广泛用于企业开发。",
        metadata={"source": "java_guide"}
    ),
]

# 创建子文档分割器
child_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)

# 创建存储
vectorstore = InMemoryVectorStore(embeddings)
store = InMemoryStore()

# 创建ParentDocumentRetriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

# 添加文档（会自动分割并存储父子关系）
parent_retriever.add_documents(parent_docs)

# 检索
query = "可读性"
docs = parent_retriever.invoke(query)

print(f"查询: {query}")
print(f"\n返回完整父文档:")
for doc in docs:
    print(f"- {doc.page_content}")
    print(f"  来源: {doc.metadata['source']}")

## 6. 自定义Retriever

使用@chain装饰器创建自定义Retriever。

In [9]:
print("【自定义Retriever】")
print()

from langchain_core.runnables import chain
from typing import List

@chain
def custom_retriever(query: str) -> List[Document]:
    """自定义检索逻辑：结合关键词和语义搜索"""
    # 1. 关键词过滤
    keyword_filtered = []
    keywords = ["Python", "Java", "JavaScript"]
    
    for keyword in keywords:
        if keyword.lower() in query.lower():
            # 基于关键词过滤文档
            keyword_filtered.extend([
                doc for doc in documents 
                if keyword in doc.metadata.get("topic", "")
            ])
    
    # 2. 如果有关键词匹配，返回过滤结果
    if keyword_filtered:
        return keyword_filtered[:3]
    
    # 3. 否则使用语义搜索
    return vector_store.similarity_search(query, k=3)

# 测试自定义retriever
query1 = "Python的特点"
docs1 = custom_retriever.invoke(query1)
print(f"查询1: {query1}")
print(f"结果: {len(docs1)} 个文档（关键词匹配）\n")

query2 = "如何处理数据"
docs2 = custom_retriever.invoke(query2)
print(f"查询2: {query2}")
print(f"结果: {len(docs2)} 个文档（语义搜索）")

【自定义Retriever】

查询1: Python的特点
结果: 2 个文档（关键词匹配）

查询2: 如何处理数据
结果: 3 个文档（语义搜索）



## 7. 检索策略对比

不同检索策略的适用场景对比。


【检索策略对比表】
检索策略对比：

| 策略 | 优势 | 劣势 | 适用场景 |
|------|------|------|----------|
| Similarity | 简单快速 | 可能返回重复相似文档 | 通用场景 |
| MMR | 平衡相关性和多样性 | 计算成本高 | 需要多样化结果 |
| Score Threshold | 过滤不相关结果 | 可能返回空结果 | 对质量要求高 |
| MultiQuery | 提高召回率 | 需要额外LLM调用 | 查询表达不清晰 |
| Compression | 减少token，提高质量 | 需要额外LLM调用 | 文档很长 |
| ParentDocument | 精确检索+完整上下文 | 实现复杂 | 层级文档结构 |

选择建议：
1. 默认使用 Similarity
2. 结果重复 → MMR
3. 需要高质量 → Score Threshold
4. 查询模糊 → MultiQuery
5. 文档太长 → Compression
6. 需要上下文 → ParentDocument
"""

## 8. 实战项目：智能检索系统

构建一个综合检索系统，根据查询类型自动选择最佳检索策略。

In [10]:
print("【智能检索系统】")
print()

class SmartRetriever:
    """智能检索系统：自动选择最佳检索策略"""
    
    def __init__(self, vector_store, model):
        self.vector_store = vector_store
        self.model = model
        
        # 预定义各种retriever
        self.retrievers = {
            "similarity": vector_store.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 3}
            ),
            "mmr": vector_store.as_retriever(
                search_type="mmr",
                search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.5}
            ),
            "threshold": vector_store.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={"score_threshold": 0.3, "k": 5}
            ),
        }
    
    def analyze_query(self, query: str) -> str:
        """分析查询，选择最佳策略"""
        query_lower = query.lower()
        
        # 规则1：包含"多个"、"不同"等词 → MMR
        if any(word in query_lower for word in ["多个", "不同", "各种", "比较"]):
            return "mmr"
        
        # 规则2：包含"最好"、"最相关" → Threshold
        if any(word in query_lower for word in ["最好", "最相关", "高质量"]):
            return "threshold"
        
        # 默认：Similarity
        return "similarity"
    
    def retrieve(self, query: str):
        """智能检索"""
        strategy = self.analyze_query(query)
        retriever = self.retrievers[strategy]
        docs = retriever.invoke(query)
        
        return {
            "query": query,
            "strategy": strategy,
            "documents": docs,
            "count": len(docs)
        }

# 创建智能检索系统
smart_retriever = SmartRetriever(vector_store, model)

# 测试不同类型的查询
test_queries = [
    "Python的特点",
    "比较不同的编程语言",
    "最相关的机器学习资料"
]

print("=" * 60)
for query in test_queries:
    result = smart_retriever.retrieve(query)
    print(f"\n查询: {result['query']}")
    print(f"策略: {result['strategy']}")
    print(f"结果数: {result['count']}")
    print(f"文档: {[doc.metadata['topic'] for doc in result['documents'][:3]]}")
    print("=" * 60)

print("\n智能检索系统演示完成！")

【智能检索系统】


查询: Python的特点
策略: similarity
结果数: 3
文档: ['Python', 'Java', 'Python']

查询: 比较不同的编程语言
策略: mmr
结果数: 3
文档: ['Java', 'Python', 'AI']


/home/iip/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1043: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'type': 'concept', 'topic': 'AI'}, page_content='机器学习是人工智能的一个分支，让计算机从数据中学习。'), 0.19842358668719418), (Document(metadata={'topic': 'Python', 'type': 'application'}, page_content='Python广泛应用于数据科学、机器学习和Web开发。'), 0.18577750908519752), (Document(metadata={'topic': 'RAG', 'type': 'concept'}, page_content='RAG技术结合检索和生成，让AI能回答私有数据问题。'), 0.13651728407002883), (Document(metadata={'type': 'concept', 'topic': 'AI'}, page_content='深度学习使用神经网络处理复杂的模式识别任务。'), 0.11591995061536087), (Document(metadata={'topic': 'Python', 'type': 'intro'}, page_content='Python是一种高级编程语言，以简洁的语法著称。python是由top开发'), -0.054883693310312065)]
  self.vectorstore.similarity_search_with_relevance_scores(



查询: 最相关的机器学习资料
策略: threshold
结果数: 0
文档: []

智能检索系统演示完成！


## 9. 总结与最佳实践

### 核心概念回顾

1. Retriever是Runnable接口，支持invoke、batch等方法
2. VectorStoreRetriever提供三种基础检索策略
3. 高级Retriever提供更智能的检索能力
4. 可以根据场景选择或组合多种检索策略

### Retriever的关键类型

**基础检索策略（VectorStoreRetriever）：**
- Similarity: 相似度检索（默认）
- MMR: 最大边际相关性（多样性）
- Score Threshold: 阈值过滤（质量保证）

**高级检索器：**
- MultiQueryRetriever: 多查询变体（提高召回）
- ContextualCompressionRetriever: 上下文压缩（减少噪音）
- ParentDocumentRetriever: 父文档检索（保留上下文）

### 最佳实践

1. **从简单开始**: 先用Similarity，发现问题再优化
2. **合理设置k值**: 通常3-5个结果，根据实际调整
3. **监控检索质量**: 记录检索结果，分析相关性
4. **选择合适策略**: 
   - 结果重复 → MMR
   - 需要高质量 → Score Threshold
   - 查询模糊 → MultiQuery
   - 文档太长 → Compression
5. **考虑性能成本**: 高级Retriever可能需要额外LLM调用
6. **A/B测试**: 对比不同策略的实际效果

### 常见问题与解决

- **检索结果太相似**: 使用MMR增加多样性
- **检索到不相关文档**: 使用Score Threshold过滤
- **检索结果太少**: 降低阈值或增加k值
- **查询表达不清**: 使用MultiQuery生成多个变体
- **文档太长**: 使用ContextualCompression压缩
- **缺少上下文**: 调整chunk_overlap或使用ParentDocument
- **检索太慢**: 减少k值或简化检索策略
- **成本太高**: 避免使用需要LLM的Retriever（MultiQuery、Compression）

### 检索策略选择指南

| 场景 | 推荐策略 | 原因 |
|------|---------|------|
| 通用场景 | Similarity | 简单快速 |
| 需要多样化结果 | MMR | 平衡相关性和多样性 |
| 对质量要求高 | Score Threshold | 过滤低质量结果 |
| 查询表达模糊 | MultiQuery | 多角度检索 |
| 文档内容冗长 | Compression | 提取关键信息 |
| 需要完整上下文 | 大chunk_overlap | 保留更多上下文 |
| 复杂业务逻辑 | 自定义Retriever | 灵活控制 |

### 性能优化建议

1. **索引优化**: 选择合适的向量存储（生产环境避免InMemory）
2. **批量处理**: 使用batch方法处理多个查询
3. **缓存策略**: 缓存常见查询的结果
4. **异步调用**: 使用ainvoke进行异步检索
5. **监控指标**: 跟踪检索延迟、相关性评分